## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.stats import norm, lognorm, uniform

In [ ]:
print("Yfinance version: " , yf.__version__)

In [ ]:
# Set default parameters for plt.title()
plt.rcParams['axes.titlepad'] = 12  # Set padding around the title
plt.rcParams['axes.titleweight'] = 'normal'  # Set title font size
plt.rcParams['axes.titlesize'] = 9  # Set title font size

## Fetch cotation data

In [ ]:
def fetch_stock_data(tickers, start_date, end_date, interval):
    data = yf.download(tickers, start=start_date, end=end_date, interval=interval)
    return data

In [ ]:
# select tickers
tickers = ["IAU", "GOOG", "MSFT", "BRK-B", "KO", "JNJ", "DG", "DIS"] #, "BRK-B", "IAU", "GOOG", "AMZN", "AAPL", "TSM", "BAC", "WFC"]  #VALE, KO, JNJ, DG, DIS, SPY 

# date format YYYY-MM-DD
start_date = "2010-01-01"
end_date = "2024-02-01"

# time interval to take the cotations, in this case monthly
interval= "1d"

stock_data = fetch_stock_data(tickers, start_date, end_date, interval)

display(stock_data)

In [ ]:
# select only the closing price
df_close = stock_data.Close.copy()
df_close.info()
df_close

## Calculate returns

### Year-over-Year Return (YoY)

In [ ]:
returns_daily = (df_close
                 .copy()
                 .pct_change(periods=252)
                 .dropna()
)

# display(returns_daily.describe())
returns_daily.head(3)

## Fit distribution model - Returns

In [ ]:
def calculate_return_params(key):
    sigma, loc, scale = lognorm.fit(returns_daily[key] + 1.0)    
    return (sigma, loc, scale)

In [ ]:
# Defining return dict to receive the fitting parameters from distributions
params = ("sigma", "loc", "scale")
return_fitting_params_lognormal = { key: dict( zip( params, calculate_return_params(key)) ) for key in tickers }

In [ ]:
# return_fitting_params_lognormal

## Return plots

### Lognormal:

In [ ]:
# Plot the histogram of the returns_daily along with the fitted Gaussian distribution
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
tick0 = tickers[0]
tick1 = tickers[2]
# Fit a Gaussian distribution to the returns_daily
plt.subplot(1, 2, 1)
data = returns_daily[tick0] + 1.0
sns.histplot(data= data, bins=30, stat="density", kde=True, color='lightgreen', edgecolor='red')

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 1000)
p = lognorm.pdf(x, return_fitting_params_lognormal[tick0]["sigma"], loc= return_fitting_params_lognormal[tick0]["loc"], scale= return_fitting_params_lognormal[tick0]["scale"])

plt.plot(x, p, 'k', label="fit-lognormal", linewidth=1)
plt.title(f"{tick0} --> Fit results: sigma = %.2f,  loc = %.2f, scale = %.2f" % (return_fitting_params_lognormal[tick0]["sigma"],
                                                                                      return_fitting_params_lognormal[tick0]["loc"],
                                                                                      return_fitting_params_lognormal[tick0]["scale"]))
plt.ylabel("Probability Density Function")
plt.xlabel("1 + return - [a.a.]")
plt.legend()


# Fit a Gaussian distribution to the returns_daily
plt.subplot(1, 2, 2)
data = returns_daily[tick1] + 1.0
sns.histplot(data= data, bins=30, stat="density", kde=True, color='lightgreen', edgecolor='red')

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 1000)
p = lognorm.pdf(x, return_fitting_params_lognormal[tick1]["sigma"], loc= return_fitting_params_lognormal[tick1]["loc"], scale= return_fitting_params_lognormal[tick1]["scale"])

plt.plot(x, p, 'k', label="fit-lognormal", linewidth=1)
plt.title(f"{tick1} --> Fit results: sigma = %.2f,  loc = %.2f, scale = %.2f" % (return_fitting_params_lognormal[tick1]["sigma"],
                                                                                      return_fitting_params_lognormal[tick1]["loc"],
                                                                                      return_fitting_params_lognormal[tick1]["scale"]))
plt.ylabel("Probability Density Function")
plt.xlabel("1 + return - [a.a.]")
plt.legend()

fig.tight_layout()
fig.subplots_adjust(wspace=0.5)
plt.show()

## Generate samples from distributions models

In [ ]:
# Number of samples
num_samples = 1_000

In [ ]:
def calculate_return_samples(ticker, num_samples=1_000):
    sigma   = return_fitting_params_lognormal[ticker]["sigma"]
    loc     = return_fitting_params_lognormal[ticker]["loc"]
    scale   = return_fitting_params_lognormal[ticker]["scale"]
    samples = lognorm.rvs(sigma, loc=loc, scale=scale, size=num_samples) - 1.0
    return samples

# Defining return dict to receive the samples from distributions models builted with their respective parameters
return_samples_lognormal = { ticker: calculate_return_samples(ticker, num_samples) for ticker in tickers }


## Plot - Compare data vs model

### Returns:

In [ ]:
# Find the best parameters when fitting the model to the data
data = returns_daily[tickers[0]] + 1.0
sigma, loc, scale = lognorm.fit(data)

# Generate samples based on the parameters find previously
n_samples = 100_000
samples = lognorm.rvs(sigma, loc=loc, scale=scale, size=n_samples)

# Plot the histogram of the generated samples
plt.figure(figsize=(10,5))
sns.histplot(data, bins=30, stat="density", color='lightblue', edgecolor='lightblue', label="data samples")
sns.histplot(samples, bins=30, stat="density", kde=True, color='lightgreen', edgecolor='lightgreen', alpha=0.3, label="lognormal samples")

# Plot the PDF of the lognormal distribution for comparison
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
pdf = lognorm.pdf(x, sigma, loc= loc, scale= scale)

plt.plot(x, pdf, 'k', linewidth=2, label="lognormal curve")
plt.title(f"{tickers[0]} --> Fit results: sigma = %.2f,  loc = %.2f, scale = %.2f" % (sigma, loc, scale))
plt.ylabel("Density")
plt.xlabel("1 + return - [a.a.]")
plt.legend()
plt.show()

## Monte carlo simulation

### Equations:
<br>

- **Portfolio Return**:
<br>
$$ R_P = {\omega}^T \cdot R $$
<br>
with $R_P$ being the return of the portfolio, $\omega$ the weights and $R$ the returns of the assets. Each one are given by:
<br><br>

<table>
  <tr>
    <td>
      $$
        \omega = \begin{bmatrix}
                {\omega}_A \\
                {\omega}_B \\
                {\omega}_C \\
        \end{bmatrix},
      $$
    </td>
    <td>
      and
    </td>
    <td>
    $$
        R = \begin{bmatrix}
                R_A \\
                R_B \\
                R_C \\
        \end{bmatrix}.
    $$
    </td>
  </tr>
</table>

<br>

- **Portfolio Risk**: 
<br>
$$ {{\sigma}^2_P} = {\omega}^T \cdot \Sigma \cdot \omega $$
<br>
in wich ${{\sigma}^2_P}$ is the variance of the portfolio and $\Sigma$ is the covariance matrix of the assets. The covariance matrix is given by:
<br>
<br>
$$ \Sigma = \begin{bmatrix}
                {{\sigma}^2_A} & cov(A,B) & cov(A,C) \\
                cov(A,B) & {{\sigma}^2_B} & cov(B,C) \\
                cov(A,C) & cov(B,C) & {{\sigma}^2_C} \\
            \end{bmatrix}.
$$
<br>




## Generate asset weights

In [ ]:
def make_wight_matrix(n_tickers, n_samples=1_000):
    rand_values = uniform.rvs(size= int(n_tickers * n_samples)).reshape((n_tickers, n_samples))
    normalized_values = rand_values / rand_values.sum(axis=0, keepdims=True)
    return normalized_values

# Matrix of the weights
weight_matrix = make_wight_matrix(len(tickers), num_samples)
weight_matrix.shape

## Build Assets Return Matrix

In [ ]:
# Create empty array with the final shape
assets_return_matrix = np.zeros((len(tickers), num_samples), dtype=np.float64)

# Map the values in the dictionary to the new array
for row, ticker in zip(range(len(tickers)), tickers):
    assets_return_matrix[row, :] = return_samples_lognormal[ticker].reshape(1, num_samples)

assets_return_matrix[:, :5]

## Portfolio Return

In [ ]:
# # Matrix of the returns
portfolio_return = weight_matrix.T.dot(assets_return_matrix).reshape(-1)
portfolio_return.shape

In [ ]:
# Plot
sns.histplot(portfolio_return, bins=50, stat="density", kde=True, color='lightgreen', edgecolor='lightgreen', alpha=0.3, label=f"Assets: {tickers}")

threshold = 0.1
return_threshold = np.count_nonzero(portfolio_return > threshold) / np.max(portfolio_return.shape)

plt.title(f"Portfolio Return - Prob(return >= {int(threshold * 100)}%) = {int(100 * return_threshold)}.0%")
plt.ylabel("Probability density function")
plt.xlabel("return [a.a.]")
plt.legend(loc="upper right")
plt.show()